<a href="https://colab.research.google.com/github/davibernardes035/FineTunningWithBert/blob/main/FineTuneBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacotes e imports

In [ ]:
!pip install -q --upgrade transformers datasets evaluate huggingface_hub wandb

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

## Tratamento dos dados

In [ ]:
df = pd.read_csv("dados.csv", encoding="utf-8", delimiter=";", on_bad_lines="skip")
df

,label,text,label_text
0,1,"""A ação ordinária foi extinta sem resolução do...",RECURSO DE APELACAO
1,2,"""A parte interessada deve interpor recurso de ...",RECURSO DE APELACAO
2,3,"""A parte deve estar presente na sessão de julg...",SESSAO DE JULGAMENTO
3,4,"""As partes têm um prazo de 5 (cinco) dias para...",MANIFESTACAO
4,5,"""O Agravo de Instrumento deve ser interposto e...",AGRAVO DE INSTRUMENTO
...,...,...,...
60,84,A parte interessada pode fazer um pedido de re...,RECURSO DE APELACAO
61,85,As partes foram notificadas de que o julgado f...,RECURSO ESPECIAL
62,86,Fica estabelecido que o exequente tem o dever ...,MANIFESTACAO
63,87,O prazo para apresentação das observações do i...,MANIFESTACAO


In [ ]:
#verificar o balanceamento dos dados
print(df['label'].value_counts())

label
2    28
0    14
6    11
4     5
7     3
1     2
3     1
5     1
Name: count, dtype: int64


Index(['label', ' text', ' label_text'], dtype='object')

In [ ]:

labels = df['label_text'].unique()

label2id = {valor: indice for indice, valor in enumerate(labels)} # Dicionario de conversao de label para id
id2label = {indice: valor for indice, valor in enumerate(labels)} # Dicionario de conversao de id para label

# Criar uma nova coluna com os números relativos à classe
df['label'] = df['label_text'].map(label2id)

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, )    # Separa 80% para dados de Treino
# val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, ) # Divide o restante em dados de Teste e Validacao (10% cada)

train_df = df

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
    "test": Dataset.from_pandas(test_df)
})

In [ ]:
#Garanta que todos os labels estao sendo treinados
unique_count = train_df['label'].unique()
print(f"Quantidade de valores únicos na coluna 'label': {unique_count}")

Quantidade de valores únicos na coluna 'label': [0 1 2 3 4 5 6 7]


# Tokenizacao dos dados

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-multilingual-cased", num_labels=len(labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True, max_length=28)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(len(tokenized_datasets["train"]))) # Embaralha todos os dados.
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(len(tokenized_datasets["test"])))

# Configuracoes de treino

In [ ]:
!wandb login 8caf33e6be26dcbb0b37ce7c5df343b31ab54360 # COnfigura a chave do WanDB

training_args = TrainingArguments(
    run_name="test_trainer",
    output_dir="trained_model",
    num_train_epochs=3,

    #report_to="none"   # Configura para nao reportar para o WanDB
    )

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from transformers import EarlyStoppingCallback, IntervalStrategy
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

args = TrainingArguments(
   output_dir = "training_with_callbacks",
   eval_strategy = IntervalStrategy.STEPS,
   eval_steps = 50,
   save_total_limit = 5,
   learning_rate=2e-5,
   per_device_train_batch_size=28,
   per_device_eval_batch_size=28,
   num_train_epochs=9,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True
   )

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)
#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred, average='micro')
#     precision = precision_score(y_true=labels, y_pred=pred, average='micro')
#     f1 = f1_score(y_true=labels, y_pred=pred)
#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

# Treino do modelo

In [ ]:
trainer.train()
#key = "8caf33e6be26dcbb0b37ce7c5df343b31ab54360"

# Avaliação do modelo

In [ ]:
metrics = trainer.evaluate()
metrics

# Exportação do Modelo

In [ ]:
model.save_pretrained("test_trainer/model")

# Recarregamento e teste do modelo salvo

In [ ]:
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("test_trainer/model")

#input = "Entrega de bens para pagamento de uma dívida"

input = dataset['test'][5]
tt=tokenizer(input['text'], return_tensors="pt", padding=True, truncation=True)


# input = "A parte deve estar presente na sessão de julgamento marcada para o dia 17 de julho de 2024, às 13:30, onde será analisada a Apelação Cível referente ao processo 0003793-28.2020.8.16.0194. A não comparecimento pode resultar em prejuízo para a parte apelante."
# tt=tokenizer(input, return_tensors="pt", padding=True, truncation=True)


model.eval() # Configura o modelo no modo de avaliacao, ativa todas as camadas

with torch.no_grad():
    outputs=model(**tt)

logits = outputs.logits
# Convert logits to probabilities using softmax
probabilities = F.softmax(logits, dim=-1)

predicted_class = torch.argmax(probabilities, dim=-1).item()

print("Entrada: ", input)
print("Saida:", predicted_class, f"- {id2label[predicted_class]}")

Entrada:  {'label': 4, 'text': 'As partes foram notificadas de que o julgado foi divulgado no ambiente eletrônico do PJe e podem acessá-lo no segundo grau de jurisdição. O prazo para apresentação de recurso é de 15 dias corridos, a partir da data de divulgação do julgado.', 'label_text': 'RECURSO ESPECIAL', '__index_level_0__': 61}
Saida: 2 - MANIFESTACAO


In [ ]:
test_df

,label,text,label_text
53,0,A parte atingida tem o direito de promover um ...,RECURSO DE APELACAO
60,0,A parte interessada pode fazer um pedido de re...,RECURSO DE APELACAO
0,0,"""A ação ordinária foi extinta sem resolução do...",RECURSO DE APELACAO
45,6,O devedor deve fornecer a demonstração atualiz...,APRESENTACAO DE CALCULOS
5,0,A parte interessada deve apresentar recurso de...,RECURSO DE APELACAO
61,4,As partes foram notificadas de que o julgado f...,RECURSO ESPECIAL
16,0,A ação comum foi declaradamente extinta por fa...,RECURSO DE APELACAO
12,5,O Recurso de Apelação deve ser interposto em r...,RECURSO DE APELAÇÃO
64,2,O reclamante tem o prazo de 08 (oito) dias sub...,MANIFESTACAO
30,2,É imprescindível que as partes exibam sua opin...,MANIFESTACAO


In [ ]:
train_df

,label,text,label_text
0,0,"""A ação ordinária foi extinta sem resolução do...",RECURSO DE APELACAO
1,0,"""A parte interessada deve interpor recurso de ...",RECURSO DE APELACAO
2,1,"""A parte deve estar presente na sessão de julg...",SESSAO DE JULGAMENTO
3,2,"""As partes têm um prazo de 5 (cinco) dias para...",MANIFESTACAO
4,3,"""O Agravo de Instrumento deve ser interposto e...",AGRAVO DE INSTRUMENTO
...,...,...,...
60,0,A parte interessada pode fazer um pedido de re...,RECURSO DE APELACAO
61,4,As partes foram notificadas de que o julgado f...,RECURSO ESPECIAL
62,2,Fica estabelecido que o exequente tem o dever ...,MANIFESTACAO
63,2,O prazo para apresentação das observações do i...,MANIFESTACAO
